In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import time
import copy
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import torchvision.models as models
from __future__ import print_function, division

cudnn.benchmark = True
plt.ion()   # interactive mode



```
# This is formatted as code
```

# Checking for computing device


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

# *Adjusting Data*

In [5]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# ***Loading Data Path***

In [8]:
data_dir = '/content/drive/MyDrive/Data/classification_dataset'


image_datasets_train = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transform[x]) for x in ['train']}

dataloaders_train = {x: torch.utils.data.DataLoader(image_datasets_train[x], batch_size=543, shuffle=True, num_workers=2) for x in ['train']}

dataset_sizes_train = {x: len(image_datasets_train[x]) for x in ['train']}

class_names_train = image_datasets_train['train'].classes

print(dataset_sizes_train)
print(class_names_train)

image_datasets_test = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transform[x]) for x in ['test']}

dataloaders_test = {x: torch.utils.data.DataLoader(image_datasets_test[x], batch_size=543, shuffle=True, num_workers=2) for x in ['test']}

dataset_sizes_test = {x: len(image_datasets_test[x]) for x in ['test']}

class_names_test = image_datasets_test['test'].classes


print(dataset_sizes_test)


{'train': 532}
['bear', 'butterfly', 'camel', 'chimp', 'duck', 'elephant']
{'test': 120}


# ***Initializing ImageNet pretrained ResNet18***

In [9]:
res18 = models.resnet18(pretrained=True)
print(res18)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# ***Extracting Feature Vectors from layers prior to Fully Connected Layers of ResNet18*** 

In [10]:
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
nodes, _ = get_graph_node_names(res18)
print(nodes)

return_nodes = {
    'flatten' : 'lay'
}

res18 = create_feature_extractor(res18, return_nodes=return_nodes)
res18.to(device)



['x', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.add', 'layer1.0.relu_1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.add', 'layer1.1.relu_1', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.0.add', 'layer2.0.relu_1', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.add', 'layer2.1.relu_1', 'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.downsample.0', 'layer3.0.downsample.1', 'layer3.0.add', 'layer3.0.relu_1', 'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu', 'layer3.1.conv2', 'layer3.1.bn2', 'layer3.1.add', 'layer3.1.relu_1', 'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.downsample.0', 'layer4.0.downsample.1', 'lay

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Module(
    (0): Module(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Module(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2):

In [11]:
res18.eval()
for param in res18.parameters():
  param.requires_grad = False

#extracting the trainfeatures
for inputs, labels in dataloaders_train['train']:
    inputs = inputs.to(device)
features = res18(inputs)
train_feature_vectors = features['lay'].cpu().detach().numpy()
# print(labels.numpy())
train_label_vectors = labels.numpy()
print(train_feature_vectors.shape, train_label_vectors.shape)

#extracting the test-features
for inputs, labels in dataloaders_test['test']:
    inputs = inputs.to(device)
features = res18(inputs)
test_feature_vectors = features['lay'].cpu().numpy()
# print(labels.numpy())
test_label_vectors = labels.numpy()
print(test_feature_vectors.shape, test_label_vectors.shape)

(532, 512) (532,)
(120, 512) (120,)


In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

res18.eval()
neigh = KNeighborsClassifier(n_neighbors = 10)
neigh.fit(train_feature_vectors, train_label_vectors) 
test_pred = neigh.predict(test_feature_vectors)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_label_vectors, test_pred)*100)

Accuracy: 95.83333333333334
